# Project - Introduction to machine learning
### Francesco Carzaniga and Sonia Donati








The purpose of the project is to find the best machine learning algorithm for a particular dataset.

The algorithms that we are going to use are: Support Vector Machine (with three different kernels), K-nearest neighbour, Artificial Neural Network and finally Random Forest (implemented by us).

## 1. Problem description


The data 'dataset32.csv' is compiled from car accidents, classified according to their severity.
* Number of samples: 499
* Number of features: 13

The considered features are as follows:

0. '**time_to_aid**': time before receiving first aid (in minutes)
1. '**time_from_road_check**': time from last road maintenance (in years)
2. '**avg_speed**': average speed at impact
3. '**road_state**': average number of injured people per vehicle
4. '**ppl_vehicle**': average number of people per vehicle
5. '**avg_time_in_care**': average time spent in hospital care per injured person
6. '**num_rescue**': number of rescuers on the scene
7. '**time_to_hospital**': time to reach the hospital (in minutes)
8. '**age_vehicles**': average age of vehicles involved
9. '**time_from_vehicle_check**': time from last vehicle safety check
10. '**road_type**': road network type (local, regional, national)

The goal is to predict the severity of an accident. 

**Remarks:** 
* '**class**': accident severity (0 = no injuries, 1 = non-fatal, 2 = fatal injuries) is not a feature
* '**vehicle_number**': vehicle registration number is not useful

## 2. Data preprocessing

First of all we have to import the necessary packages.

In [26]:
import numpy as np
import pandas as pd

from joblib import Parallel, delayed

from sklearn.base import BaseEstimator, ClassifierMixin, MetaEstimatorMixin
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

import xgboost as xgb

Now, we load the dataset using pandas.

In [3]:
dataset = pd.read_csv('dataset32.csv', delimiter = ";").values

This way we obtain our dataset as a numpy array. \
The last column contain the classes (i.e. 0, 1, 2), we call it *y*:

In [4]:
y = dataset[:,13]

Moreover, it is important to notice that the three classes are well balanced, as is shown below:

In [5]:
unique, counts = np.unique(y, return_counts=True)
print([counts[i]/np.sum(counts) for i in range(len(counts))])

[0.33867735470941884, 0.34468937875751504, 0.3166332665330661]


We reshape *y* and create a copy:

In [6]:
y = y.astype(np.float).reshape((dataset.shape[0],1))

y_Rf = np.copy(y) # for Random forest (see Chapter 4)

Finally, we can select the features. In this case, we omit the last and third-last column of the dataset (see Remarks in the previous chapter). Again we also create a copy that will maintain the categorical features for later use.

In [7]:
dataset = dataset[:,[0,1,2,3,4,5,6,7,8,9,10,12]]
print(dataset.shape)

dataset_Rf = np.copy(dataset) # dataset with strings for Random forest (see Chapter 4)

(499, 12)


Unfortunately, the dataset presents some strings and missing values. In what follows, we transform these strings to integers.
We cannot however perform the imputation already, as it would leak information from the test to the train subsets we will introduce later. We have to split first and then do imputation!

In [8]:
le = preprocessing.LabelEncoder()
dataset[:,3] = le.fit_transform(dataset[:,3])  # 'road_state': average = 0, bad = 1, good = 2
dataset[:,11] = le.fit_transform(dataset[:,11])  # 'road_type': local = 0 , national = 1, regional = 2

dataset = np.asarray(dataset, dtype=np.float64)  # all values of the dataset are float now

# dataset has 10 NaN values, where and how many?
for i in range(12):
    if any(np.isnan(dataset[:,i])):
       print("Feature", i, "has", sum(np.isnan(dataset[:,i])), "NaN value(s)")

Feature 6 has 4 NaN value(s)
Feature 7 has 6 NaN value(s)


As usual, we shuffle the data.

In [9]:
def shuffle(dataset, y):
    z = np.hstack((dataset, y))
    np.random.shuffle(z)
    return np.hsplit(z, [dataset.shape[1]])

dataset, y = shuffle(dataset, y)

dataset_Rf, y_Rf = shuffle(dataset_Rf, y_Rf)

## 3. Model implementations

In this chapter we will implement all the methods for the project. We have a multiclass classification problem, but we want to reduce it to multiple binary decisions. In order to do that, we write two Python classes that transform the task to either a OneVsOne or a OneVsAll problem.

**OneVsOne**

In [10]:
class OneVsOne(BaseEstimator, ClassifierMixin, MetaEstimatorMixin):
    def __init__(self, model=None, n_jobs=-1, **parameters): # initialize self 
        self.model = model
        self.n_jobs = n_jobs
        self.parameters = parameters
        self.classes = None
        self.model_list = None

    def get_params(self, deep=True): # get parameters
        return {**{"model": self.model}, **{"n_jobs": self.n_jobs}, **self.parameters}

    def __fit_ovo_estimator(self, X, y, class_one, class_two): # transform the models into 0 vs 1
        class_selection = np.logical_or(y == class_one, y == class_two)
        current_model = self.model().set_params(**self.parameters)
        y = y[class_selection]
        y_binarized = np.zeros_like(y)
        y_binarized[y == class_one] = 0
        y_binarized[y == class_two] = 1
        X = X[class_selection]
        current_model.fit(X, y_binarized)
        return current_model, class_one, class_two

    def fit(self, X, y): # use parallel implementation to fit estimator for each pair of classes
        self.classes = np.unique(y)
        models = Parallel(n_jobs=self.n_jobs)(delayed(self.__fit_ovo_estimator)
                                              (X, y, self.classes[i], self.classes[j]) for i in range(len(self.classes))
                                              for j in range(i + 1, len(self.classes)))
        self.model_list = list(zip(*models))
        return

    @staticmethod
    def __predict_ovo_estimator(X, model):
        return model.predict(X)

    @staticmethod
    def __predict_proba_ovo_estimator(X, model): 
        # the method predict_proba or decision_function, already present in the models,
        # outputs which model is the most confident 
        try:
            confidence = np.max(model.predict_proba(X), axis=1)
        except (AttributeError, NotImplementedError):
            confidence = model.decision_function(X)
        return confidence

    def predict(self, X): 
        # predict from a certain model the best class for every label in X
        # if there are possible ties between the models, 
        # the function takes the class with the most confidence
        models = self.model_list[0]
        predictions = np.stack(Parallel(n_jobs=self.n_jobs)(delayed(self.__predict_ovo_estimator)(X, models[i])
                                                            for i in range(len(models)))).astype(dtype=np.int32).T
        confidences = np.stack(Parallel(n_jobs=self.n_jobs)(delayed(self.__predict_proba_ovo_estimator)(X, models[i])
                                                            for i in range(len(models)))).T
        votes = np.zeros((X.shape[0], self.classes.size))
        total_confidences = np.zeros_like(votes)
        for model in range(len(models)):
            class_one_m = self.model_list[1][model]
            class_two_m = self.model_list[2][model]
            votes[predictions[:, model] == 0, np.argwhere(self.classes == class_one_m)[0]] += 1
            votes[predictions[:, model] == 1, np.argwhere(self.classes == class_two_m)[0]] += 1
            total_confidences[predictions[:, model] == 0, np.argwhere(self.classes == class_one_m)[0]] += \
                confidences[predictions[:, model] == 0, model]
            total_confidences[predictions[:, model] == 1, np.argwhere(self.classes == class_two_m)[0]] += \
                confidences[predictions[:, model] == 1, model]
        # trasform confidences between [-1/3, 1/3] to avoid overriding the votes, clever trick taken from sklearn
        transformed_confidences = (total_confidences /
                                   (3 * (np.abs(total_confidences) + 1))) 
        winners = self.classes[np.argmax(votes+transformed_confidences, axis=1)]
        return winners

**OneVsAll**

In [11]:
class OneVsAll(object):
    def __init__(self, model=None, n_jobs=-1, **parameters): # initialize self
        self.model = model
        self.model_list = []
        self.n_jobs = n_jobs
        self.classes = None
        self.parameters = parameters

    def get_params(self, deep=True): # get parameters
        return {**{"model": self.model}, **{"n_jobs": self.n_jobs}, **self.parameters}
 
    def set_params(self, **parameters): # set parameters
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def __fit_ova_estimator(self, X, y, class_one): # trasform all the models into 1 vs 0 (class_one is 1, the rest 0)
        current_model = self.model().set_params(**self.parameters)
        y_binarized = np.zeros_like(y)
        y_binarized[y == class_one] = 1
        y_binarized[y != class_one] = 0
        current_model.fit(X, y_binarized)
        return current_model, class_one
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        models = Parallel(n_jobs=self.n_jobs)(delayed(self.__fit_ova_estimator)
                                              (X, y, self.classes[i]) for i in range(len(self.classes)))
        self.model_list = list(zip(*models))
        return
    
    @staticmethod
    def __predict_ova_estimator(X, model):
        return model.predict(X)

    @staticmethod
    def __predict_proba_ova_estimator(X, model):
        try:
            confidence = np.max(model.predict_proba(X), axis=1)
        except (AttributeError, NotImplementedError):
            confidence = model.decision_function(X)
        return confidence
    
    def predict(self, X):
        # predict from a certain model the best class for every label in X
        # if there are possible misunderstanding between the models, 
        # the function take the class given by the most confident model
        models = self.model_list[0]
        predictions = np.stack(Parallel(n_jobs=self.n_jobs)(delayed(self.__predict_ova_estimator)(X, models[i])
                                                            for i in range(len(models)))).astype(dtype=np.int32)
        confidences = np.stack(Parallel(n_jobs=self.n_jobs)(delayed(self.__predict_proba_ova_estimator)(X, models[i])
                                                            for i in range(len(models))))
        
        val = []
        for k in range(X.shape[0]):
            index = np.argwhere(predictions[:,k] == 1)
            if index.size == 1: # if there is a unique 1 in the column k
                val.append(self.classes[index])
            elif index.size == 0: # if there are none
                conf = confidences[:,k]
                val.append(self.classes[np.argmax(conf)])
            else:
                conf = np.multiply((predictions[:, k] + confidences[:, k]), (predictions[:, k]))  # add the confidence only to the values with 1
                val.append(self.classes[np.argmax(conf)])
        val_array = np.asarray(val)
        return val_array

    def score(self, X, y): # this function returns the accuracy of the prediction given X and y
        label_predict = self.predict(X)
        loss = np.mean(y.ravel() == label_predict)
        return loss

We wanted to implement the solutions in two different ways, in fact the first one vectorizes over the samples while the second one over the models. The tie breaking decisions are taken in the standard way.

Last thing to implement is the Random forest algorithm.

First we build a tree class that serves as a basic data structure for our model. On top of it we build a decision tree using the C4.5 algorithm, and finally we construct the random forest from multiple decision trees.

**Rmk:** 
* C4.5 algorithm is different from the one used in sklearn, which is CART
* Advantage: C4.5 algorithm supports both numerical and categorical values, while CART does not.

**Tree**

In [13]:
class Tree(object):
    def __init__(self, parent=None, children=None, feature=None, threshold=None, direction=None, excluded_samples=None,
                 is_leaf=False, decision=None, confidence=None):
        if children is None:
            children = []
        self.parent = parent
        self.children = children
        self.feature = feature
        self.threshold = threshold
        self.direction = direction
        self.excluded_samples = excluded_samples
        self.is_leaf = is_leaf
        self.decision = decision
        self.confidence = confidence
        self.depth = self.compute_depth()

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def compute_depth(self): # compute the depth of the tree
        depth = 0
        node = self
        while node.parent is not None:
            node = node.parent
            depth += 1
        return depth

    def add_child(self, child): # append element to children list
        self.children.append(child)

    def get_parent(self):
        return self.parent

    def get_children(self):
        return self.children

    def get_feature(self):
        return self.feature

    def get_threshold(self):
        return self.threshold

    def get_all_features(self): # get the features of this node and all the parents
        node = self
        features_list = []
        while node is not None:
            features_list.append(int(node.get_feature()))
            node = node.parent
        return np.asarray(features_list)

    def get_direction(self):
        return self.direction

    def set_parent(self, parent):
        self.parent = parent

    def set_feature(self, feature):
        self.feature = feature

    def set_threshold(self, threshold):
        self.threshold = threshold

    def set_direction(self, direction):
        self.direction = direction

    def __max_depth(self, tree): # depth of the tree starting from this node
        if tree.is_leaf:
            return 0
        elif len(tree.children) == 0:
            return 0
        else:
            depth = []
            for child in tree.children:
                depth.append(self.__max_depth(child))
            return np.amax(depth)+1.

    def get_max_depth(self):
        return self.__max_depth(self)

    def get_depth(self):
        return self.depth

    def set_excluded_samples(self, excluded_samples):
        self.excluded_samples = excluded_samples
        return

    def get_all_excluded_samples(self): # get the samples excluded by this node and all the parents
        node = self
        samples_list = np.asarray([])
        while node is not None:
            samples_list = np.concatenate([samples_list, node.get_excluded_samples().ravel()])
            node = node.parent
        return np.asarray(samples_list)

    def get_is_leaf(self):
        return self.is_leaf

    def set_is_leaf(self, is_leaf):
        self.is_leaf = is_leaf

    def get_decision(self):
        return self.decision

    def set_decision(self, decision):
        self.decision = decision

    def get_excluded_samples(self):
        return np.asarray(self.excluded_samples)

    def get_confidence(self):
        return self.confidence

**Decision Tree**

In [14]:
class DecisionTree(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None, max_features=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.tree = None
        self._queue = []
        self.classes = None

    @staticmethod
    def __entropy(labels): # compute entropy
        if labels.size == 0:
            return 0
        unique, counts = np.unique(labels, return_counts=True)
        return np.sum([-counts[i]/np.sum(counts)*np.log2(counts[i]/np.sum(counts)) for i in range(len(unique))])

    def __gain(self, y, subsets):  
    # normalized information gain of a feature, so that we can choose the most efficient one to use
    # i.e. (initial entropy of all labels -  entropy of labels with some chosen feature)
        entropy_node = self.__entropy(y)
        total_length = np.sum([subset.size for subset in subsets], dtype=np.float64)
        weights = [subset.size/total_length for subset in subsets]
        entropy_child = np.sum([weights[i]*self.__entropy(y[subsets[i]]) for i in range(len(subsets))])
        return entropy_node-entropy_child

    def __split(self, X, y, node, excluded_samples=None, direction=None):
    # Choose remaining features and samples to be tested
        dataset_size, label_size = X.shape
        if excluded_samples is None:
            excluded_samples = []
        if node is not None:
            excluded_features = node.get_all_features()
            features = np.delete(np.arange(label_size), excluded_features)
            all_excluded_samples = node.get_all_excluded_samples()
            all_excluded_samples = np.concatenate([all_excluded_samples, excluded_samples]).astype(dtype=np.int32)
            samples = np.delete(np.arange(dataset_size), all_excluded_samples)
        else:
            features = np.arange(label_size)
            samples = np.arange(dataset_size)
        y_orig = np.copy(y)
        X = X[samples]
        y = y[samples]
        classes, counts = np.unique(y, return_counts=True)
        confidence = np.zeros(2)
        # Base case 1, labels are all the same so create leaf where decision is label
        if classes.size == 1:
            confidence[np.argwhere(self.classes == classes[0])] = 1.
            leaf = Tree(parent=node, decision=classes[0], direction=direction, is_leaf=True, confidence=confidence)
            node.add_child(leaf)
            return 1
        # Base case 2, no labels associated to this class so decide with the most frequent label of parent
        elif classes.size == 0:
            unique, counts = np.unique(y_orig[samples], return_counts=True)
            total_labels = np.sum(counts)
            for u in range(unique.size):
                confidence[np.argwhere(self.classes == unique[u])] = counts[u]/total_labels
            all_excluded_samples = node.get_all_excluded_samples().astype(dtype=np.int32)
            samples = np.delete(np.arange(dataset_size), all_excluded_samples)
            leaf = Tree(parent=node, decision=int(np.median(y_orig[samples]).round()), direction=direction,
                        is_leaf=True, confidence=confidence)
            node.add_child(leaf)
            return 2
        # Max depth parameter must be respected
        if self.max_depth is not None and node is not None and node.get_max_depth() == self.max_depth - 1:
            unique, counts = np.unique(y, return_counts=True)
            total_labels = np.sum(counts)
            for u in range(unique.size):
                confidence[np.argwhere(self.classes == unique[u])] = counts[u]/total_labels
            leaf = Tree(parent=node, decision=int(np.median(y).round()), direction=direction, is_leaf=True,
                        confidence=confidence)
            node.add_child(leaf)
            return 4
        # Max_features must be respected
        max_features = self.max_features
        if max_features is not None and features.size > max_features:
            random_features = np.random.choice(features, max_features, replace=False)
        else:
            random_features = features
            
        # Try all the chosen features. Obtain feature with best gain and its threshold.
        max_gain = 0.
        max_feature = -1
        best_threshold = None
        for feature in random_features:
            feature_vector = X[:, feature]
            try:
                feature_vector = np.array(feature_vector, dtype=np.float64) # if numerical convert to float64
            except ValueError:
                feature_vector = np.array(feature_vector, dtype=object) # if categorical convert to object
            unique, counts = np.unique(feature_vector, return_counts=True)
            if feature_vector.dtype == 'object': # categorical case
                subsets = [np.argwhere(feature_vector == u) for u in unique]
                gain = self.__gain(y, subsets)
                threshold = None
            elif feature_vector.dtype == 'float64': # numerical case
                threshold_gains = []
                for u in unique:
                    below = np.argwhere(feature_vector <= u)
                    above = np.argwhere(feature_vector > u)
                    threshold_gains.append(self.__gain(y, [below, above]))
                gain = np.nanmax(threshold_gains) # take maximal gain
                threshold = unique[np.nanargmax(threshold_gains)]
            if gain > max_gain:
                max_gain = gain
                max_feature = feature
                best_threshold = threshold
        # Base case 3. Can no longer get better, make parent a leaf.
        if max_gain == 0.:
            unique, counts = np.unique(y_orig[samples], return_counts=True)
            total_labels = np.sum(counts)
            for u in range(unique.size):
                confidence[np.argwhere(self.classes == unique[u])] = counts[u] / total_labels
            all_excluded_samples = node.get_all_excluded_samples().astype(dtype=np.int32)
            samples = np.delete(np.arange(dataset_size), all_excluded_samples)
            new_node = Tree(parent=node.parent, decision=int(np.median(y_orig[samples]).round()),
                            direction=node.get_direction(), is_leaf=True, confidence=confidence)
            substitute = node.parent.children.index(node)
            node.parent.children[substitute] = new_node
            return 3
        # Create new node with best feature. If there is entropy gain, create new node (node not leaf!).
        new_node = Tree(parent=node, direction=direction, feature=max_feature, threshold=best_threshold,
                        excluded_samples=excluded_samples)
        if node is not None:
            node.add_child(new_node)
        return new_node

    def __create_nodes_numerical(self, X, y, feature_vector, node_thresh, node):
        less = np.argwhere(feature_vector <= node_thresh).ravel()
        great = np.argwhere(feature_vector > node_thresh).ravel()
        case = self.__split(X, y, node, great, 'l')
        if isinstance(case, Tree):
            self._queue.append(case)
        elif case == 3:
            return
        case = self.__split(X, y, node, less, 'g')
        if isinstance(case, Tree):
            self._queue.append(case)
        elif case == 3:
            return

    def __create_nodes_categorical(self, X, y, feature_vector, unique, node):
        for u in unique:
            excluded_samples = np.argwhere(feature_vector != u).ravel()
            case = self.__split(X, y, node, excluded_samples, u)
            if isinstance(case, Tree):
                self._queue.append(case)
            elif case == 3:
                return

    def fit(self, X, y): # build tree breadth-wise
        if self.max_features is None:
            self.max_features = X.shape[1]
        self.classes = np.unique(y)
        self.tree = self.__split(X, y, self.tree)
        self._queue.append(self.tree)
        while len(self._queue) > 0:
            node = self._queue.pop()
            node_feat = node.get_feature()
            node_thresh = node.get_threshold()
            feature_vector = X[:, node_feat]
            unique, counts = np.unique(feature_vector, return_counts=True)
            if node_thresh is None:
                self.__create_nodes_categorical(X, y, feature_vector, unique, node)
            else:
                self.__create_nodes_numerical(X, y, feature_vector, node_thresh, node)
        return

    def predict(self, X): # traverse the tree and take leaf decision
        prediction = []
        for sample in X:
            node = self.tree
            while not node.is_leaf:
                feature = node.get_feature()
                threshold = node.get_threshold()
                if threshold is None:
                    value = sample[feature]
                    children_direction = [child.direction for child in node.children]
                    direction = children_direction.index(value)
                    node = node.children[direction]
                else:
                    if sample[feature] - threshold < 0:
                        direction = 0
                    else:
                        direction = 1
                    node = node.children[direction]
            prediction.append(node.get_decision())
        return np.asarray(prediction)

    def predict_proba(self, X): # for the confidence (used in OneVsOne, OneVsAll)
        proba = []
        for sample in X:
            node = self.tree
            while not node.is_leaf:
                feature = node.get_feature()
                threshold = node.get_threshold()
                if threshold is None:
                    value = sample[feature]
                    children_direction = [child.direction for child in node.children]
                    direction = children_direction.index(value)
                    node = node.children[direction]
                else:
                    if sample[feature] - threshold < 0:
                        direction = 0
                    else:
                        direction = 1
                    node = node.children[direction]
            proba.append(node.get_confidence())
        return np.asarray(proba)

**Random Forest**

In [15]:
class RandomForest(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=None, max_features=None, n_estimators=10, bootstrap=1., n_jobs=-1):
        self.max_depth = max_depth
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.n_jobs = n_jobs
        self.bootstrap = bootstrap
        self._estimators = []

    def __make_estimators(self): # build trees in a parallel way
        estimators = Parallel(n_jobs=self.n_jobs)\
            (delayed(DecisionTree)(max_depth=self.max_depth, max_features=self.max_features)
             for i in range(self.n_estimators))
        return estimators

    @staticmethod
    def __parallel_build_trees(tree, X, y, bootstrap): # build single tree
        if bootstrap:
            samples = np.random.choice(np.arange(X.shape[0]), int(bootstrap*X.shape[0]))
            X = X[samples]
            y = y[samples]
        tree.fit(X, y)
        return tree

    def fit(self, X, y):
        estimators = self.__make_estimators()
        result = Parallel(n_jobs=self.n_jobs)\
            (delayed(self.__parallel_build_trees)(tree, X, y, self.bootstrap) for tree in estimators)
        self._estimators = result
        return

    def predict(self, X):
        results = Parallel(n_jobs=self.n_jobs)(delayed(element.predict)(X) for element in self._estimators)
        return np.median(np.stack(results), axis=0).round()

    def predict_proba(self, X):
        results = Parallel(n_jobs=self.n_jobs)(delayed(element.predict_proba)(X) for element in self._estimators)
        return np.mean(np.stack(results, axis=2), axis=2)

## 4. Validation and Testing

Afterwards, we do 5-fold cross-validation for all the models. First with OneVsOne, then with OneVsAll.

**Rmk:** Cross-validation is a more robust method than the classic splitting into train and test sets. 

To avoid clutter, we first initialize the imputer that will be used for all the computations (except RandomForest). The reason is explained in Section 2.

In [16]:
median_imputer = SimpleImputer(missing_values = np.nan, strategy="median")

**SVM (Default: Kernel "rbf")**

In [18]:
model_SVM = svm.SVC

# do imputation and trasform the multiclass problem to OnevsOne
onevone_transform = OneVsOne(model_SVM, gamma = "auto")
estimator_ovo = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

# do cross-validation
val_ovo = cross_validate(estimator_ovo, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OnevsAll

onevall_transform = OneVsAll(model_SVM, gamma = "auto")
estimator_ova = Pipeline([("Imputer", median_imputer),("Transformer", onevall_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,2.446348,4.040210,0.850000
1,1.887950,3.642261,0.800000
2,1.874985,3.655622,0.800000
3,1.863017,4.050170,0.880000
4,1.970728,3.784879,0.808081


This is the mean of the test_score with OneVsOne: 0.8276161616161616


,fit_time,score_time,test_score
0,1.955773,3.920513,0.860000
1,1.957763,4.033213,0.820000
2,1.879976,3.878627,0.800000
3,2.515272,4.628646,0.880000
4,2.347476,4.476092,0.808081


This is the mean of the test_score with OneVsAll: 0.8336161616161617


**Rmk:** 
* This result with gamma = "auto" is much better than with gamma = "scale" 
* In the version of sklearn 0.22 the default parameter gamma changes from "auto" to "scale".

**Polynomially kernelized SVM**

In [19]:
model_poly = svm.SVC

onevone_transform = OneVsOne(model_poly, kernel = 'poly')
estimator_ovo = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator_ovo, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OnevsAll

onevall_transform = OneVsAll(model_poly, kernel = 'poly')
estimator_ova = Pipeline([("Imputer", median_imputer),("Transformer", onevall_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,2.129307,3.592395,0.84000
1,1.870994,4.113002,0.71000
2,1.944801,4.371306,0.69000
3,2.342738,3.789866,0.81000
4,1.924849,3.814113,0.69697


This is the mean of the test_score with OneVsOne: 0.7493939393939394


,fit_time,score_time,test_score
0,1.891940,3.892627,0.83000
1,1.902872,3.891593,0.86000
2,2.045531,3.808815,0.81000
3,1.968734,3.815796,0.87000
4,1.974719,3.901600,0.79798


This is the mean of the test_score with OneVsAll: 0.8335959595959597


**Linear kernelized SVM**

In [20]:
model_linear = svm.SVC

onevone_transform = OneVsOne(model_linear, kernel = "linear")
estimator = Pipeline([("imputer", median_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OneVsAll

onevall_transform = OneVsAll(model_linear, kernel = "linear")
estimator_ova = Pipeline([("imputer", median_imputer),("Transformer", onevall_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,2.105370,3.847524,0.880000
1,2.126311,4.092057,0.870000
2,1.919864,3.789636,0.860000
3,1.924852,3.964400,0.940000
4,1.797193,3.688656,0.818182


This is the mean of the test_score with OneVsOne: 0.8736363636363637


,fit_time,score_time,test_score
0,1.962791,3.911503,0.900000
1,1.990685,3.742982,0.870000
2,2.031565,3.722048,0.860000
3,2.007625,3.857684,0.920000
4,2.027575,3.788869,0.838384


This is the mean of the test_score with OneVsAll: 0.8776767676767676


**K-nearest neighbour algorithm**

In [22]:
model_K = KNeighborsClassifier

onevone_transform = OneVsOne(model_K,n_neighbors=5)
estimator_ovo = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator_ovo, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OneVsAll

onevall_transform = OneVsAll(model_K,n_neighbors=5)
estimator_ova = Pipeline([("Imputer", median_imputer),("Transformer", onevall_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,2.117334,3.882620,0.850000
1,1.869998,4.367321,0.760000
2,2.932159,4.438134,0.810000
3,2.014611,4.020250,0.830000
4,1.927847,3.934477,0.777778


This is the mean of the test_score with OneVsOne: 0.8055555555555556


,fit_time,score_time,test_score
0,2.106365,3.995318,0.790000
1,1.878975,3.930489,0.730000
2,2.138284,4.407213,0.790000
3,2.073456,3.962404,0.810000
4,2.012618,4.005848,0.767677


This is the mean of the test_score with OneVsAll: 0.7775353535353535


**Artificial neural network**

In [23]:
model_ANN = MLPClassifier

onevone_transform = OneVsOne(model_ANN)
estimator = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OneVsAll

onevall_transform = OneVsAll(model_ANN)
estimator_ova = Pipeline([("imputer", median_imputer),("Transformer", onevall_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,2.506298,4.104029,0.860000
1,2.125316,3.473711,0.880000
2,2.222054,3.743991,0.810000
3,2.121327,3.740001,0.950000
4,2.111387,3.847677,0.828283


This is the mean of the test_score with OneVsOne: 0.8656565656565658


,fit_time,score_time,test_score
0,2.406600,3.696083,0.830000
1,2.261952,3.697110,0.880000
2,2.439478,3.746982,0.770000
3,2.237015,3.906554,0.890000
4,2.339747,3.744982,0.757576


This is the mean of the test_score with OneVsAll: 0.8255151515151515


### Random forest

We use first the dataset with categorical values, then with numerical values. The algorithm is very similar in both cases, but it is still interesting to observe.

NB: in the former case we use the imputation strategy "most_frequent", since it is impossible to compute the mean with categorical values inside the dataset.

**Categorical Case**

In [24]:
model_Rf = RandomForest
categorical_imputer = SimpleImputer(missing_values = np.nan, strategy = "most_frequent")

onevone_transform = OneVsOne(model_Rf)
estimator_ovo = Pipeline([("Imputer", categorical_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator_ovo, dataset_Rf, y_Rf.astype(np.float).ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OneVsAll

onevall_transform = OneVsAll(model_Rf)
estimator_ova = Pipeline([("Imputer", categorical_imputer),("Transformer", onevall_transform)])

val_ova = cross_validate(estimator_ova, dataset_Rf, y_Rf.astype(np.float).ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,24.166067,4.089506,0.900000
1,25.468426,4.298285,0.850000
2,24.655653,4.052078,0.890000
3,24.578756,3.953024,0.840000
4,23.994878,4.116960,0.858586


This is the mean of the test_score with OneVsOne: 0.8677171717171717


,fit_time,score_time,test_score
0,41.123852,4.769299,0.850000
1,55.145694,5.037460,0.910000
2,42.054918,4.558225,0.820000
3,40.295504,4.599126,0.830000
4,43.844697,4.989157,0.767677


This is the mean of the test_score with OneVsAll: 0.8355353535353535


**Numerical case**

In [25]:
model_Rf = RandomForest

onevone_transform = OneVsOne(model_Rf)
estimator_ovo = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator_ovo, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OneVsAll

onevall_transform = OneVsAll(model_Rf)
estimator_ova = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,28.369689,4.388958,0.890000
1,27.738514,3.828265,0.890000
2,26.612033,4.037036,0.820000
3,27.182295,3.990066,0.890000
4,24.176726,4.261467,0.808081


This is the mean of the test_score with OneVsOne: 0.8596161616161616


,fit_time,score_time,test_score
0,26.580022,3.786295,0.85000
1,25.182423,3.762829,0.87000
2,24.759525,3.761971,0.86000
3,25.129227,3.783057,0.90000
4,23.500609,3.776826,0.79798


This is the mean of the test_score with OneVsAll: 0.8555959595959596


For curiosity's sake we also try XGBoost, which is currently the leading machine learning algorithm in terms of performance.

In [27]:
model_xgb = xgb.XGBClassifier

onevone_transform = OneVsOne(model_xgb)
estimator_ovo = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ovo = cross_validate(estimator_ovo, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ovo))
mean_val_ovo = val_ovo["test_score"].mean()
print("This is the mean of the test_score with OneVsOne:", mean_val_ovo)

# Now the same with OneVsAll

onevall_transform = OneVsAll(model_xgb)
estimator_ova = Pipeline([("Imputer", median_imputer),("Transformer", onevone_transform)])

val_ova = cross_validate(estimator_ova, dataset, y.ravel(), cv=5)

display(pd.DataFrame(val_ova))
mean_val_ova = val_ova["test_score"].mean()
print("This is the mean of the test_score with OneVsAll:", mean_val_ova)

,fit_time,score_time,test_score
0,2.194874,4.386270,0.870000
1,2.228040,4.512076,0.880000
2,2.313812,4.657213,0.880000
3,2.476341,5.541183,0.880000
4,2.908222,5.804480,0.818182


This is the mean of the test_score with OneVsOne: 0.8656363636363636


,fit_time,score_time,test_score
0,2.498078,4.866985,0.870000
1,2.400582,4.686468,0.880000
2,2.361686,4.792182,0.880000
3,2.364677,4.731938,0.880000
4,2.375649,4.847079,0.818182


This is the mean of the test_score with OneVsAll: 0.8656363636363636


## 5. Conclusion

From what we have obtained in the previous chapters, we can draw the following conclusions.
The best algorithm for the given dataset is the linear kernelized SVM, while slightly below it are tied ANN, Random Forest, and XGBoost. This means that very likely the dataset has been generated linearly with some perturbations, with the SVM capturing the linearity best (as expected) and the others trying to model the random noise, hence overfitting in the end.
Moreover the difference between OneVsOne and OneVsAll is not great, indeed with some algorithms the former works more efficiently, while with others the converse is true.

**Remark:** In this context hyperparameter optimization would produce no benefit. Multiple models with very different characteristics and properties all hit a performance wall around 0.87, meaning that it is extremely unlikely that any tinkering would improve the result.